# Follow_Line Practice

### API Summary Table
| Description                                   | Method                                                       |
| --------------------------------------------- | ------------------------------------------------------------ |
| Get the images from the camera                | - `input_image = self.getImage()`                            |
| Move the robot                                | - `self.motors.sendW(5)` <br />-  `self.motors.sendV(10)`          |
| Change the image inRGB to HSV                 | - `image_HSV = cv2.cvtColor()`                               |
| Filter the red values                         | - `value_min_HSV = np.array([0, 235, 60])`<br />- `value_max_HSV = np.array([180, 255, 255])` |
| Filter the images                             | - `image_HSV_filtered = cv2.inRange()`                       |
| Create a mask with the red values             | - `image_HSV_filtered_Mask = np.dstack(())`                  |
| Get the numbers of the image rows and columns | - `size = input_image.shape`                                 |
| Get the pixels that change of tone            | - `position_pixel_left = []`<br />- `position_pixel_right = []` |
| Calculate the desviation of the car           | -   `desviation = position_middle - (columns/2)`             |
| Save the camera image                         | - `self.set_color_image(input_image)`                        |
| Save the filtered image                       | - `self.set_threshold_image(image_HSV_filtered_Mask)`        |
| Show an image                                 | - `printImage(img)`                                          |      

### Exercise initialization
---
First of all, we need to run the Gazebo simulator. The environment and the robot models are automatically launched, as you can see in the right side of the screen. If you are not seeing something similar to the image shown below, ensure to zoom out a little bit to seek for them:

<img src="world.png">

Second, we need to import the necessary files. Once done, we need to call `Follow_Line` class once. Run this code and wait a few seconds until follow line initialization finishes with an OK message:

In [ ]:
#!/usr/bin/python
#-*- coding: utf-8 -*-

from local_follow_line import FollowLine, printImage
%matplotlib inline

fl = FollowLine()
fl.play()

Now we can start coding to give intelligence to the Formula1 robot. We can do it modifying the `execute()` method from Follow Line component. This method will be called iteratively. Let's try it with a dummy instruction: each iteration, we'll print a message.

In [ ]:
# Implement execute method
def execute(self):
    print "Running execute iteration"
      
fl.setExecute(execute)

Stop printing the updating (message) of the method with an empty instruction:

In [ ]:
def execute(self):
    pass
    # your code goes here
fl.setExecute(execute)

Now you have seen how the `execute` method materializes the changes in the `FollowLine` component, just include there your code. Cheer Up!

In [ ]:
import cv2
import numpy as np
def execute(self):
    #GETTING THE IMAGES
    input_image = self.getImage()

    # RGB model change to HSV
    image_HSV = cv2.cvtColor(input_image, cv2.COLOR_RGB2HSV)

    # Minimum and maximum values of the red ​​
    value_min_HSV = np.array([0, 235, 60])
    value_max_HSV = np.array([180, 255, 255])

    # Filtering images
    image_HSV_filtered = cv2.inRange(image_HSV, value_min_HSV, value_max_HSV)

    # Creating a mask with only the pixels within the range of red
    image_HSV_filtered_Mask = np.dstack((image_HSV_filtered, image_HSV_filtered, image_HSV_filtered))

    # Shape gives us the number of rows and columns of an image
    size = input_image.shape
    rows = size[0]
    columns = size[1]

    #  Looking for pixels that change of tone
    position_pixel_left = []
    position_pixel_right  = []

    for i in range(0, columns-1):
        value = image_HSV_filtered[365, i] - image_HSV_filtered[365, i-1]
        if(value != 0):
            if (value == 255):
                position_pixel_left.append(i)
            else:
                position_pixel_right.append(i-1)

    # Calculating the intermediate position of the road
    if ((len(position_pixel_left) != 0) and (len(position_pixel_right) != 0)):
        position_middle = (position_pixel_left[0] + position_pixel_right[0]) / 2
    elif ((len(position_pixel_left) != 0) and (len(position_pixel_right) == 0)):
        position_middle = (position_pixel_left[0] + columns) / 2
    elif ((len(position_pixel_left) == 0) and (len(position_pixel_right) != 0)):
        position_middle = (0 + position_pixel_right[0]) / 2
    else:
        position_pixel_right.append(1000)
        position_pixel_left.append(1000)
        position_middle = (position_pixel_left[0] + position_pixel_right[0])/ 2

    # Calculating the desviation
    desviation = position_middle - (columns/2)
    print ("desviation: ", desviation)

    #EXAMPLE OF HOW TO SEND INFORMATION TO THE ROBOT ACTUATORS
    if (desviation == 0):
         self.motors.sendV(3)
    elif (position_pixel_right[0] == 1000):
         self.motors.sendW(-0.35)
    elif ((abs(desviation)) < 85):
         if ((abs(desviation)) < 15):
             self.motors.sendV(1)
         else:
             self.motors.sendV(3.5)
         self.motors.sendW(-0.00045 * desviation)
    elif ((abs(desviation)) < 150):
         if ((abs(desviation)) < 120):
             self.motors.sendV(1)
         else:
             self.motors.sendV(1)
         self.motors.sendW(-0.00045 * desviation)
    else:
         self.motors.sendV(1)
         self.motors.sendW(-0.0055 * desviation)

    # Saving the filtered image
    self.set_threshold_image(image_HSV_filtered_Mask)
        
fl.setExecute(execute)

### Just a few Examples and Tips

To obtain an image of the camera, the data is first saved and then displayed. You can use these instructions to display the contents of the camera.

In [ ]:
def execute(self):
    img = self.getImage()
    self.set_color_image(img)
    printImage(img)
    
fl.setExecute(execute)

You can use the following instructions to show the filtered images:

In [ ]:
def execute(self):
    img = self.getImage()
    filtered_img = None
    # image processing...
    # filtered_img = foo
    self.set_threshold_image(filtered_img)
    printImage(filtered_img)
    
fl.setExecute(execute)

In [ ]:
def execute(self):
    segmentedImage = fl.get_threshold_image()
    printImage(segmentedImage)
    
fl.setExecute(execute)